In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import  DecisionTreeRegressor
from sklearn.metrics import precision_score,roc_auc_score,mean_squared_error,d2_absolute_error_score,accuracy_score
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/student-performance-predictions/st.ipynb
/kaggle/input/student-performance-predictions/student_performance.csv
/kaggle/input/student-performance-predictions/best_student_performance_model.pkl


In [2]:
data=pd.read_csv("/kaggle/input/student-performance-predictions/student_performance.csv")
data.head()

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade
0,1,John,Male,85,15,78,1,High,80
1,2,Sarah,Female,90,20,85,2,Medium,87
2,3,Alex,Male,78,10,65,0,Low,68
3,4,Michael,Male,92,25,90,3,High,92
4,5,Emma,Female,88,18,82,2,Medium,85


In [3]:
data=data.drop(columns=['StudentID','Name','Gender'])
data.head()

,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade
0,85,15,78,1,High,80
1,90,20,85,2,Medium,87
2,78,10,65,0,Low,68
3,92,25,90,3,High,92
4,88,18,82,2,Medium,85


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   AttendanceRate             10 non-null     int64 
 1   StudyHoursPerWeek          10 non-null     int64 
 2   PreviousGrade              10 non-null     int64 
 3   ExtracurricularActivities  10 non-null     int64 
 4   ParentalSupport            10 non-null     object
 5   FinalGrade                 10 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 608.0+ bytes


In [5]:
ohe=OneHotEncoder(sparse=False)
encoded_columns=ohe.fit_transform(data[['ParentalSupport']])
encoded_col_names=ohe.get_feature_names_out(['ParentalSupport'])
encoded_df=pd.DataFrame(encoded_columns,columns=encoded_col_names)
data=pd.concat([data,encoded_df],axis=1)

In [6]:
data=data.drop('ParentalSupport',axis=1)

In [7]:
data.head()

,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,FinalGrade,ParentalSupport_High,ParentalSupport_Low,ParentalSupport_Medium
0,85,15,78,1,80,1.0,0.0,0.0
1,90,20,85,2,87,0.0,0.0,1.0
2,78,10,65,0,68,0.0,1.0,0.0
3,92,25,90,3,92,1.0,0.0,0.0
4,88,18,82,2,85,0.0,0.0,1.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   AttendanceRate             10 non-null     int64  
 1   StudyHoursPerWeek          10 non-null     int64  
 2   PreviousGrade              10 non-null     int64  
 3   ExtracurricularActivities  10 non-null     int64  
 4   FinalGrade                 10 non-null     int64  
 5   ParentalSupport_High       10 non-null     float64
 6   ParentalSupport_Low        10 non-null     float64
 7   ParentalSupport_Medium     10 non-null     float64
dtypes: float64(3), int64(5)
memory usage: 768.0 bytes


In [9]:
X=data.drop('FinalGrade',axis=1)
y=data['FinalGrade']

In [10]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=55)
print('x_train shape: ',x_train.shape)
print('x_test.shape: ',x_test.shape)
print('y_train.shape: ',y_train.shape)
print('y_test.shape: ',y_test.shape)

x_train shape:  (8, 7)
x_test.shape:  (2, 7)
y_train.shape:  (8,)
y_test.shape:  (2,)


In [11]:
rfr=RandomForestRegressor()
model=rfr.fit(x_train,y_train)
model.score(x_test,y_test)

0.7807200000000021

In [12]:
model.predict([[75,25,50,2,1,0,1]])


array([78.27])

In [13]:
prediction = rfr.predict(x_test)
mse = mean_squared_error(y_test, prediction)
rmse = mse**.5
print(mse)
print(rmse)

1.3704999999999874
1.1706835610018564
